In [ ]:
import pandas as pd
import glob
import datetime

In [ ]:
datapath = './Data'

In [ ]:
files = glob.glob(datapath+'/*chain*.json')
files.sort()

In [ ]:
def extract_openinterest(data_in):
    option_type = ['callExpDateMap', 'putExpDateMap']
    data_out = pd.DataFrame(columns=['optiontype', 'date', 'strike', 'openinterest'])
    for option in option_type:
        dates = data_in[option].keys()
        for date in dates:
            openinterest = []
            strikes = data_in[option][date].keys()
            for strike in strikes:
                openinterest.append(data_in[option][date][strike][0]['openInterest'])
            temp = pd.DataFrame()
            temp['openinterest'] = openinterest
            temp['strike'] = strikes
            temp['date'] = date.split(':')[0]
            temp['optiontype'] = option[:-10]
            data_out = data_out.append(temp)
    return data_out

In [ ]:
data_in = pd.read_json(files[-1])
data_out = extract_openinterest(data_in)

In [ ]:
top_calls = data_out[data_out['optiontype']=='call'].sort_values('openinterest', ascending=False)
top_calls = top_calls[top_calls['openinterest']>=1000].reset_index(drop=True)

In [ ]:
top_puts = data_out[data_out['optiontype']=='put'].sort_values('openinterest', ascending=False)
top_puts = top_puts[top_puts['openinterest'] >= 1000].reset_index(drop=True)

In [ ]:
top_options = data_out.sort_values('openinterest', ascending=False).head(10).reset_index(drop=True)

In [ ]:
data_in = pd.read_json(files[-2])
d2 = extract_openinterest(data_in)

In [ ]:
data_out['name'] = data_out.apply(lambda x: (x.optiontype+'_'+x.date+'_'+x.strike), axis=1)
d2['name'] = d2.apply(lambda x: (x.optiontype+'_'+x.date+'_'+x.strike), axis=1)
data_out = data_out[['name', 'openinterest']]
d2 = d2[['name', 'openinterest']]
merged_df = pd.merge(data_out, d2, on='name', suffixes=("_before", "_after"))
merged_df['change'] = merged_df.apply(lambda x: x.openinterest_after-x.openinterest_before, axis=1)

In [ ]:
top_delta = merged_df.sort_values('change', ascending=False).head(10).reset_index(drop=True)

In [ ]:
bottom_delta = merged_df.sort_values('change', ascending=True).head(10).reset_index(drop=True)

In [ ]:
data = [top_calls, top_puts, top_options, top_delta, bottom_delta]
subjects = ['Top Calls', 'Top Puts', 'Top Options', 'Top Delta', 'Bottom Delta']

In [ ]:
def configure_mail(data, subject):
    config = pd.read_json('./Data/config.json', typ='series')
    recipients = pd.read_json('./Data/mail_to.json', typ='series')
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d")
    botmail = config["botmail"]
    botpw = config["botpw"]
    subject = 'AMZN OI Report - {} - '.format(subject) + timestamp
    for name, email in recipients.items():
        recipient = email
        send_email(botmail, botpw, email, subject, data)

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_email(botmail, botpw, recipient, subject, data):    
    msg = MIMEMultipart('alternative')
    msg['Subject'] = subject
    msg['From'] = botmail
    msg['To'] = recipient

    html = data.to_html()
    msg.attach(MIMEText(html, 'html'))

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()
    server.login(botmail, botpw)

    server.sendmail(botmail, recipient, msg.as_string())
    server.quit()

In [ ]:
for i in range(len(subjects)):
    configure_mail(data[i], subjects[i])